In [ ]:
import pandas as pd
import numpy as np

def get_features_num_regression(df, target_col, umbral_corr, pvalue=None):
    """
    Devuelve una lista de variables numéricas cuya correlación con la variable target es alta.

    Esta versión usa la matriz de correlación de pandas y no calcula el p-valor.
    Si se indica 'pvalue', se muestra un aviso de que no se está usando.
    
    Argumentos:
    df (pd.DataFrame): DataFrame con los datos.
    target_col (str): Columna objetivo (target) del modelo de regresión.
    umbral_corr (float): Valor entre 0 y 1 para el umbral de correlación.
    pvalue (float, opcional): No se usa en esta versión. Solo incluido para compatibilidad.

    Retorna:
    lista or None: Lista con nombres de columnas numéricas que cumplen el umbral, o None si hay errores.
    """

    # Validaciones de entrada
    if not isinstance(target_col, str) or target_col not in df.columns:
        print("Error: 'target_col' debe ser una columna válida del DataFrame.")
        return None
    if not isinstance(umbral_corr, (float, int)) or not (0 <= umbral_corr <= 1):
        print("Error: 'umbral_corr' debe estar entre 0 y 1.")
        return None
    if pvalue is not None:
        print("Aviso: Esta versión de la función no calcula p-valores. Se ignorará 'pvalue'.")

    # Comprobar que la columna target es numérica
    if not np.issubdtype(df[target_col].dtype, np.number):
        print("Error: La columna target debe ser numérica.")
        return None

    # Selección de columnas numéricas
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    if target_col not in numeric_cols:
        print("Error: La columna target no es numérica.")
        return None

    numeric_cols.remove(target_col)

    # Calculamos la correlación con la columna target
    correlaciones = df[numeric_cols + [target_col]].corr()[target_col]

    # Filtramos columnas con correlación absoluta mayor que el umbral
    columnas_seleccionadas = correlaciones[abs(correlaciones) >= umbral_corr].index.tolist()

    # Eliminamos la columna target si ha quedado dentro
    if target_col in columnas_seleccionadas:
        columnas_seleccionadas.remove(target_col)

    return columnas_seleccionadas

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

def plot_features_num_regression(df, target_col="", columns=[], umbral_corr=0.0, pvalue=None):
    """
    Genera pairplots de las columnas numéricas más correlacionadas con una variable objetivo.

    La función filtra las variables numéricas en base a un umbral de correlación y, opcionalmente,
    a un nivel de significación estadística. Si la lista de columnas es larga, divide la visualización
    en grupos de hasta 5 columnas (incluyendo siempre la variable target).
    
    Argumentos:
    df (pd.DataFrame): DataFrame con los datos.
    target_col (str): Nombre de la variable objetivo (target).
    columns (list, opcional): Lista de nombres de columnas a considerar (por defecto, se usan todas las numéricas).
    umbral_corr (float): Umbral de correlación mínima (valor absoluto) para incluir una columna.
    pvalue (float, opcional): Nivel de significación estadística. Si se proporciona, se usa test de Pearson.

    Retorna:
    lista or None: Lista de columnas que cumplen con los criterios o None si hay error.
    """

    # Validaciones de entrada
    
    if not isinstance(target_col, str) or target_col not in df.columns:
        print("Error: 'target_col' debe ser una columna existente del DataFrame.")
        return None
    if not isinstance(columns, list):
        print("Error: 'columns' debe ser una lista.")
        return None
    if not isinstance(umbral_corr, (float, int)) or not (0 <= umbral_corr <= 1):
        print("Error: 'umbral_corr' debe estar entre 0 y 1.")
        return None
    if pvalue is not None and (not isinstance(pvalue, float) or not (0 < pvalue < 1)):
        print("Error: 'pvalue' debe ser un float entre 0 y 1 o None.")
        return None
    if not np.issubdtype(df[target_col].dtype, np.number):
        print("Error: La columna 'target_col' debe ser numérica.")
        return None

    # Si columns está vacía, tomamos todas las variables numéricas excepto la target
    if not columns:
        columns = df.select_dtypes(include=np.number).columns.drop(target_col).tolist()
    else:
        # Filtramos columnas que existen en el DataFrame y son numéricas
        columns = [col for col in columns if col in df.columns and np.issubdtype(df[col].dtype, np.number)]

    columnas_filtradas = []

    for col in columns:
        try:
            correlacion = df[[target_col, col]].corr().iloc[0, 1]
            if abs(correlacion) >= umbral_corr:
                if pvalue is not None:
                    _, pval = pearsonr(df[target_col].dropna(), df[col].dropna())
                    if pval <= (1 - pvalue):
                        columnas_filtradas.append(col)
                else:
                    columnas_filtradas.append(col)
        except Exception as e:
            print(f"Advertencia: No se pudo calcular la correlación entre {target_col} y {col}. Error: {e}")

    if not columnas_filtradas:
        print("Ninguna columna cumple los criterios de correlación y significación.")
        return []

    # Dividir columnas en grupos de hasta 4 + target (máximo 5 por gráfico)
    max_columnas_por_plot = 4
    for i in range(0, len(columnas_filtradas), max_columnas_por_plot):
        grupo = columnas_filtradas[i:i + max_columnas_por_plot]
        sns.pairplot(df[[target_col] + grupo].dropna())
        plt.suptitle(f"Correlación con {target_col} (grupo {i // max_columnas_por_plot + 1})", y=1.02)
        plt.tight_layout()
        plt.show()

    return columnas_filtradas